In [1]:
import json
import typing as t
from tqdm import tqdm
import pandas as pd
import glob

In [30]:
#input_path = 'raw_data/raw_metadata/work_of_art/batch_1.json'


input_paths = glob.glob('raw_data/raw_metadata/work_of_art/*')

full_list = []
for input_path in tqdm(input_paths):

    with open(input_path) as f:
        data = f.read()

    dict_data = json.loads(data)
    #dict_data = dict_data[:10000]
    dict_data = [x for x in dict_data if x != None]
    full_list.append(dict_data)

100%|█████████████████████████████████████████████████████████████████████| 37/37 [02:29<00:00,  4.03s/it]


In [31]:
flattened_list = [item for sublist in full_list for item in sublist]

In [33]:
def data_selection_json(dict_data: t.List[t.List[t.Dict]]):

    # First clean and keep needed data
    final_list = []
    for element in tqdm(dict_data):
        
        try:

            final_ind_list = []

            for data in element:

                new_dict = dict()
                for key in data.keys():
                    new_dict[key] = data[key]['value']

                transformed_data = {}
                for key in list(new_dict.keys()):
                    if 'Label' in key:
                        non_label_key = key.split('Label')[0]
                        transformed_data[non_label_key] = {'wiki_id':new_dict[non_label_key], 'label':new_dict[key]}

                    # Handle Non Labeled keys
                    elif 'inception' in key:
                        transformed_data[key] = new_dict[key]

                final_ind_list.append(transformed_data)
                
        except:
            pass

        final_list.append(final_ind_list)
        
    return final_list

def data_cleaning_json(final_list: t.List[t.List[t.Dict]]):

    # Simplify and get rid of duplicates
    end_of_the_game_list = []
    for element in tqdm(final_list):

        # Instantiate keys
        keys = []
        for x in element:
            for key in x.keys():
                keys.append(key)
        keys = list(set(keys))

        clean_list = dict()
        for key in keys:
            clean_list[key] = list()

        for dict_element in element:
            for key in list(dict_element.keys()):
                clean_list[key].append(dict_element[key])

        # Drop Duplicates
        new_clean_list = dict()
        for key in clean_list.keys():
            d = clean_list[key]
            
            if key == 'inception':
                new_d = list(pd.DataFrame(d).drop_duplicates()[0])
            else:
                new_d = pd.DataFrame(d).drop_duplicates().to_dict(orient = 'records')
            new_clean_list[key] = new_d
        
        # subject is always unique, so we take the first element of the list
        new_clean_list['subject'] = new_clean_list['subject'][0] 
        end_of_the_game_list.append(new_clean_list)

    return end_of_the_game_list

In [34]:
def clean_date(raw_date):
    try:
        if raw_date.startswith("-"):
            clean_date = int(raw_date[:5])
        else:
            clean_date = int(raw_date[:4])

    except:
        clean_date = None
    return clean_date

In [35]:
from typing import List, Optional
from pydantic import BaseModel


class WikidataEntity(BaseModel):
    wiki_id: str
    label: str
    
    @property
    def get_id(self) -> str:
        return self.wiki_id.split("/")[-1]

class ComplexityObject(BaseModel):
    subject: WikidataEntity
    instance: Optional[List[WikidataEntity]]
    subclass: Optional[List[WikidataEntity]]
    inception: Optional[List[str]]
    time_period: Optional[List[WikidataEntity]]
    culture: Optional[List[WikidataEntity]]
    architecture_style: Optional[List[WikidataEntity]]
    founded_by: Optional[List[WikidataEntity]]
    creator: Optional[List[WikidataEntity]]
    country: Optional[List[WikidataEntity]]
    territory: Optional[List[WikidataEntity]]
    genre: Optional[List[WikidataEntity]]
    movement: Optional[List[WikidataEntity]]
    
    @property
    def get_year(self) -> str:
        return [clean_date(y) for y in self.inception]
    

In [36]:
# Clean Json

final_list = data_selection_json(flattened_list)
#final_list = final_list[:10000]

100%|████████████████████████████████████████████████████████| 3629042/3629042 [02:03<00:00, 29290.65it/s]


In [41]:

final_list = data_cleaning_json(final_list)

# Insert into objects
final_obj = [ComplexityObject(**x) for x in final_list]

100%|████████████████████████████████████████████████████████| 3629042/3629042 [1:04:43<00:00, 934.56it/s]


In [44]:
from extraction.sys_utils import save_model
save_model(final_obj, name="raw_data/work_of_art.json")

#### To Sqlite3

In [49]:
import sqlite3
import polars as pl

In [ ]:
# Take a long time for more than 1,000,000 data
df_fil = [
    {"subject":x.subject.get_id,
        "subjectLabel": x.subject.label,
        "inception": x.get_year if x.inception is not None else [],
        "instance": [y.get_id for y in x.instance] if x.instance is not None else [],
         "subclass": [y.get_id for y in x.instance] if x.subclass is not None else [],
        "creator": [y.get_id for y in x.creator] if x.creator is not None else [],
          "time_period": [y.get_id for y in x.time_period] if x.time_period is not None else [],
          "culture": [y.get_id for y in x.culture] if x.culture is not None else [],
          "architecture_style": [y.get_id for y in x.architecture_style] if x.architecture_style is not None else [],
          "founded_by": [y.get_id for y in x.founded_by] if x.founded_by is not None else [],
          "country": [y.get_id for y in x.country] if x.country is not None else [],
          "territory": [y.get_id for y in x.territory] if x.territory is not None else [],
          "genre": [y.get_id for y in x.genre] if x.genre is not None else [],
          "movement": [y.get_id for y in x.movement] if x.movement is not None else [],
        
    }
    for x in final_obj
]


In [ ]:
len(df_fil)

In [100]:
## Add only keys

# connect to SQLite3 database
conn = sqlite3.connect('works.db')

# create a cursor object
cursor = conn.cursor()

# get a list of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# drop each table in the database
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]}")

# commit the changes and close the connection
conn.commit()

In [104]:
cols = [
 'inception',
 'instance',
 'subclass',
 'creator',
 'time_period',
 'culture',
 'architecture_style',
 'founded_by',
 'country',
 'territory',
 'genre',
 'movement']

In [111]:
for col in tqdm(cols):
    print(col)
    
    if col == 'inception':
        continue
    
    df_fil_label = [
            {"subject":x.subject.get_id,
            "subjectLabel": x.subject.label,
            col: [y.get_id for y in getattr(x, col)] if getattr(x, col)is not None else [],


        }
        for x in final_obj
    ]
    
    res = pd.DataFrame(df_fil_label)    
    # only keep the subject with a proper name
    res = res[~res['subjectLabel'].str.startswith('Q')]
    res = res.explode(col)
    res = res.dropna()
    res = res[res[col].str.startswith('Q')]
    res = res.drop_duplicates()
    res.to_sql(f'subject_{col}', conn, if_exists = 'replace', index=False)

  0%|                                                                              | 0/12 [00:00<?, ?it/s]

inception
instance


 17%|███████████▌                                                         | 2/12 [10:10<50:52, 305.22s/it]

subclass


 25%|█████████████████▎                                                   | 3/12 [13:34<39:27, 263.08s/it]

...

 83%|████████████████████████████████████████████████████████▋           | 10/12 [57:30<13:17, 398.51s/it]

genre


 92%|████████████████████████████████████████████████████████████▌     | 11/12 [1:01:25<05:48, 348.66s/it]

movement


100%|██████████████████████████████████████████████████████████████████| 12/12 [1:04:27<00:00, 322.26s/it]


In [120]:
# add inception
col = 'inception'
df_fil_label = [
    {"subject":x.subject.get_id,
    "subjectLabel": x.subject.label,
    "inception": x.get_year if x.inception is not None else []



}
for x in final_obj
]

res = pd.DataFrame(df_fil_label)    
# only keep the subject with a proper name
res = res[~res['subjectLabel'].str.startswith('Q')]
res = res.explode(col)
res = res.dropna()
res = res.drop_duplicates()

res.to_sql(f'subject_{col}', conn, if_exists = 'replace', index=False)

In [123]:
all_attributes = ['subject',
                  'creator', 
                 'instance',
                  'subclass',
                 'time_period',
                 'culture',
                 'architecture_style',
                 'founded_by',
                 'country',
                 'territory',
                 'genre',
                 'movement']

for attr_to_filter in tqdm(all_attributes):
        
    name = attr_to_filter + "Label"

    final_obj_fil = [x for x in final_obj if getattr(x, attr_to_filter)]
    
    if attr_to_filter == 'subject':
    
        df_fil_label = [
                {
                    attr_to_filter: getattr(x, attr_to_filter).get_id,
                     name: getattr(x, attr_to_filter).label


                }
                for x in final_obj_fil
            ]
        
    else:
        
        df_fil_label = [
            {
                attr_to_filter: [y.get_id for y in getattr(x, attr_to_filter)] if getattr(x, attr_to_filter)is not None else [],
                 name: [y.label for y in getattr(x, attr_to_filter)] if getattr(x, attr_to_filter) is not None else [],


            }
            for x in final_obj_fil
        ]

    # insert with the count of variables
    df_fil_label = pd.DataFrame(df_fil_label)
    df_fil_label = df_fil_label.explode([attr_to_filter, name])
    df_fil_label = df_fil_label[df_fil_label[attr_to_filter].str.startswith('Q')]
    df_fil_label = df_fil_label.groupby([attr_to_filter, name])[name].count().rename('count_all').reset_index()
    df_fil_label = df_fil_label.sort_values('count_all', ascending=False)
    
    if attr_to_filter == 'subject':
        df_fil_label = df_fil_label[~df_fil_label[name].str.startswith('Q')]
        df_fil_label['origin'] = 'work_of_art_Q838948'
    
    #df_fil_label = df_fil_label.drop_duplicates().reset_index(drop=True)
    df_fil_label.to_sql(name, conn, if_exists = 'replace', index=False)

#conn.close()

100%|████████████████████████████████████████████████████████████████████| 12/12 [23:40<00:00, 118.37s/it]
